<a href="https://colab.research.google.com/github/KaniaruIvy/Compiler-Construction/blob/main/Python_to_C_Compiler_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

118751-Amugo Tatyana

115774-Kaniaru Ivy

118465- Maccine Quincy

120190-Lusiji Lyvia

120304-Owira Keith

# Python to C compiler!
from  Python
i.e 
```
(11+(9-3)  
```  
to C
```
add(11, subtract(9, 3))
```


In [175]:
#Make Necessary Imports
import copy
##regular expression library (search pattern)
import re

# There are the parts of a compiler we shall tackle
## 1 PARSING
a. Lexical Analysis
b. Syntactic Analysis/Parsing (grammar)

## 2 TRANSFORMATION
a. Tree Traversal 
b. Transform via Traversal 

## 3 CODE GENERATION


# Tokenization
The Tokenizer function receives starting input e.g (11+(9-3)) 

In [277]:
def tokenizer(input_expression):
    #counter variable for iterating through input array 
    current = 0
    #array to store computed tokens
    tokens = []
    ##use regex library to create search patterns for
    #letters a,z
    alphabet = re.compile(r"[a-z]", re.I);
    #numbers 1-9
    numbers = re.compile(r"[0-9]");
    #white space
    whiteSpace = re.compile(r"\s");
    #iterate through input
    while current < len(input_expression):
        #track position
        char = input_expression[current]
        #If white space is detected, no token created
        if re.match(whiteSpace, char):
            current = current+1
            continue
        #create + add token to array for open parens
        if char == '(':
            tokens.append({
                'type': 'left_paren',
                'value': '('
            })
            #continue iterating
            current = current+1
            continue
        #create + add token to array for closed parens
        if char == ')':
            tokens.append({
                'type': 'right_paren',
                'value': ')'
            })
            #continue iterating
            current = current+1
            continue
        #create + add token to array for numbers
        if re.match(numbers, char):
            value = ''
            #nested iteration if a number is multi-num 
            while re.match(numbers, char):
                value += char
                current = current+1
                char = input_expression[current];
            tokens.append({
                'type': 'number',
                'value': value
            })
            continue
        #create + add token to array for letters
        if char == '+':
           value = '+'
           while char == '+':
              current = current+1
              char = input_expression[current]
           tokens.append({
                'type': 'name',
                'value': value
            })
            #continue iterating
           continue
        if char == '-':
            value='-'
            while char == '-':
              current = current+1
              char = input_expression[current]
            tokens.append({
                'type': 'name',
                'value': value
            })
            #continue iterating
            continue
        if char == '*':
            value='*'
            while char == '*':
              current = current+1
              char = input_expression[current]
            tokens.append({
                'type': 'name',
                'value': value
            })
            #continue iterating
            continue
        if char == '/':
            value='/'
            while char == '/':
              current = current+1
              char = input_expression[current]
            tokens.append({
                'type': 'name',
                'value': value
            })
            #continue iterating
            continue
        #error condition if we find an unknown value in the input
        raise ValueError('Sorry, input character is unknown: ' + char);
    return tokens

# The parse function creates an Abstract Syntax Tree given the computed tokens from the previous function   

In [242]:
def parser(tokens):
    #keep track of position while iterating
    global current
    current = 0
    #nested function for building an abstract syntax tree
    def walk():
        #keep track of position while iterating
        global current
        token = tokens[current]
        #if a number is encountered, return a "NumberLiteral" node
        if token.get('type') == 'number':
          if tokens[current+1].get('type') =='name':
            current=current+2
          else:
            current=current+1
          return {
            'type': 'NumberLiteral',
            'value': token.get('value')
          }
        if token.get('type') == 'name':
          current = current + 1
          return {
              'type': 'CallExpression',
              'value': token.get('value')
            }
         
        #if open parentheses encountered, return a "CallExpression" node
        if token.get('type') == 'left_paren':
           #skip past the parenthesis, we're not storing that
            current = current + 2
            token = tokens[current]
            #store the name of operation
            node = {
                'type': 'CallExpression',
                'name': token.get('value'),
                'params': []
            }
            current = current-1
            token = tokens[current]
            #This node will have child nodes as parameters until the expression ends with a closed parens
            while token.get('type') != 'right_paren':
                
                #recursively add nodes to the params array via the walk function
                node['params'].append(walk());
                token = tokens[current]
            current = current + 1
            return  node
        #error if unknown type encountered
        raise TypeError(token.get('type'))
    
    
    #Let's initialize an empty Abstract Syntax Tree
    ast = {
        'type': 'Program',
        'body': []
    }
    #then populate it by calling the walk function
    #until the global current variable reaches the end of the token array
    while current < len(tokens):
        ast['body'].append(walk())
    #return the completed AST
    return ast

#Helper function for the transformer that enables traversing the newly created AST

In [243]:
def traverser(ast, visitor):

    #we take the child node (current AST) and parent node (new AST) as
    #inputs
    def traverseArray(array, parent):
       #iterate through every parameter element in our current AST
        for child in array:
            #and traverse each
            traverseNode(child, parent)
    
    #Take the child node (current AST) and parent node (new AST) as inputs. 
    def traverseNode(node, parent):
        method = visitor.get(node['type'])
        if method:
            method(node, parent)
        if node['type'] == 'Program':
            #traverse the body
            traverseArray(node['body'], node)
        #if its a call expression
        elif node['type'] == 'CallExpression':
            #traverse the nested paramemeters
            traverseArray(node['params'], node)
        #if its a number literal
        elif node['type'] == 'NumberLiteral':
          #break
            0
        else:
          #error for unknown type 
            raise TypeError(node['type'])
    traverseNode(ast, None)

#Using the newly created traversal functions, we can transform our existing AST for code generation

In [244]:
def transformer(ast):
   #Define an empty new AST
    newAst = {
        'type': 'Program',
        'body': []
    }
    #Copy the old one and fill the new one with it
    oldAst = ast
    ast = copy.deepcopy(oldAst)
    #let's store a reference to the newAST's body in this context property
    ast['_context'] = newAst.get('body')

    #helper function when a call expression is encountered 
    def CallExpressionTraverse(node, parent):
        expression = {
            'type': 'CallExpression',
            'callee': {
                'type': 'Identifier',
                'name': node['name']
            },
            'arguments': []
        }
        #set the current context to its child args
        node['_context'] = expression['arguments']

        #store nested call expression references
        if parent['type'] != 'CallExpression':
            expression = {
                'type': 'ExpressionStatement',
                'expression': expression
            }
        #store the expression in the context property
        parent['_context'].append(expression)

    #helper function used when a number literal is encountered during traversal. 
    def NumberLiteralTraverse(node, parent):
        parent['_context'].append({
            'type': 'NumberLiteral',
            'value': node['value']
        })
    #traverse through the AST using our helper functions until the new AST is fully populated
    traverser( ast , {
        'NumberLiteral': NumberLiteralTraverse,
        'CallExpression': CallExpressionTraverse 
    })
    #return the new AST
    return newAst

# Code generation:
A recursive stringify function that iterates through the newly created AST, node by node, continually building a string output given the values in each node.

In [271]:
def codeGenerator(node):
    if node['type'] == 'Program':
        return '\n'.join([code for code in map(codeGenerator, node['body'])])
    elif node['type'] == 'Identifier':
        return node['name']
    elif node['type'] == 'NumberLiteral':
        return node['value']
    elif node['type'] == 'ExpressionStatement':
        expression = codeGenerator(node['expression']) 
        return '%s;' % expression
    elif node['type'] == 'CallExpression':
        callee = codeGenerator(node['callee']) 
        if callee=='+':
          callee="add"
        if callee=='-':
          callee="subtract"
        if callee=='*':
          callee="multiply"
        if callee=='/':
          callee="divide"
        params = ', '.join([code for code in map(codeGenerator, node['arguments'])])
        return "%s(%s)" % (callee, params)
    else:
        raise TypeError(node['type'])

#Putting it all together

In [278]:
input = "(11*(9/3))"

In [279]:
#given an input expression, create a set of tokens
tokens = tokenizer(input)
#create an abstract syntax tree given those tokens
ast= parser(tokens)
#create a transformed AST given the existing one
newAst = transformer(ast)
#stringify the transformed AST into an output expression
output = codeGenerator(newAst)

In [280]:
print("The tokens of the input are: \n")
tokens

The tokens of the input are: 



[{'type': 'left_paren', 'value': '('},
 {'type': 'number', 'value': '11'},
 {'type': 'name', 'value': '*'},
 {'type': 'left_paren', 'value': '('},
 {'type': 'number', 'value': '9'},
 {'type': 'name', 'value': '/'},
 {'type': 'number', 'value': '3'},
 {'type': 'right_paren', 'value': ')'},
 {'type': 'right_paren', 'value': ')'}]

In [281]:
print("The Abstract Syntax Tree is:")
ast

The Abstract Syntax Tree is:


{'type': 'Program',
 'body': [{'type': 'CallExpression',
   'name': '*',
   'params': [{'type': 'NumberLiteral', 'value': '11'},
    {'type': 'CallExpression',
     'name': '/',
     'params': [{'type': 'NumberLiteral', 'value': '9'},
      {'type': 'NumberLiteral', 'value': '3'}]}]}]}

In [282]:
print("The transformed AST:")
newAst 

The transformed AST:


{'type': 'Program',
 'body': [{'type': 'ExpressionStatement',
   'expression': {'type': 'CallExpression',
    'callee': {'type': 'Identifier', 'name': '*'},
    'arguments': [{'type': 'NumberLiteral', 'value': '11'},
     {'type': 'CallExpression',
      'callee': {'type': 'Identifier', 'name': '/'},
      'arguments': [{'type': 'NumberLiteral', 'value': '9'},
       {'type': 'NumberLiteral', 'value': '3'}]}]}}]}

In [283]:
print("The original simple arithmetic python code is:"+ input+ "\n" + "It's implementation in C is:"+output)

The original simple arithmetic python code is:(11*(9/3))
It's implementation in C is:multiply(11, divide(9, 3));
